<a href="https://colab.research.google.com/github/strangeworks/examples/blob/master/examples/nec/tsp.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

In [ ]:
%pip install -U pip && pip install strangeworks-optimization

In [ ]:
import sys
import math
from itertools import product
import os
from dotenv import load_dotenv
import strangeworks as sw
from strangeworks_optimization import StrangeworksOptimizer
from strangeworks_optimization_models.problem_models import QuboDict
from strangeworks_optimization_models.parameter_models import NECParameterModel
from pyqubo import Array, Placeholder

# Load environment variables
load_dotenv()
# Authenticate with Strangeworks
sw.authenticate(api_key=os.getenv("SW_API_KEY"))

# Functions

In [ ]:
# Read TSPLIB file
def read_tsp_file(input_file):
    pos_list = []
    with open(input_file, mode='r', encoding="utf-8") as fin:
        for line in fin:
            line = line.strip()
            line2 = line.split()

            if line2[0] == 'DIMENSION':
                point_num = int(line2[2])
            if line2[0] == 'DIMENSION:':
                point_num = int(line2[1])
            if str.isdigit(line2[0]):
                pos_list.append([float(line2[1]), float(line2[2])])

    return [point_num, pos_list]

# Distance calculation between two points
def point_len(point_x1, point_y1, point_x2, point_y2):
    length2 = (point_x1 - point_x2)**2 + (point_y1 - point_y2)**2
    length = round(math.sqrt(length2))
    return length

# Distance calculation between all points
def calc_distance(pos_list, point_num):
    pos_len = []
    dlength = []

    max_len = 0
    d_min = []  # Distance array of closest cities

    for i in range(point_num):
        min_len = 10000
        for j in range(point_num):
            point_length = point_len(pos_list[i][0], pos_list[i][1], pos_list[j][0], pos_list[j][1])
            pos_len.append(point_length)
            if max_len < point_length:
                max_len = point_length
            if (min_len > point_length) and (point_length != 0):
                min_len = point_length
        dlength.append(pos_len)
        pos_len = []
        d_min.append(min_len)

    return [dlength, d_min, max_len]

# Get and check of result spins
def get_item(sample, point_num):
    result = []
    err = 0
    check = [0] * point_num
    for i in range(point_num):
        count = 0
        for j in range(point_num):
            if int(sample['x[%d][%d]' % (i, j)]) == 1:
                result.append(j)
                check[j] += 1
                count += 1
        if count != 1:
            print('ERROR : Visit %d location %d times, at time %d.' % (j, count, i))
            err = 1

    for i in range(point_num):
        if check[i] == 0:
            print('ERROR : Does not visit location[%d].' % i)
            err = 1
        elif check[i] != 1:
            print('ERROR : Visit location[%d] %d times.' % (i, check[i]))
            err = 1

    if err:
        raise ValueError('Invalid path in result.')

    return result

# Result distance calculation
def calc_length(item, pos_list):
    length = 0
    for i in range(len(item) - 1):
        length += point_len(pos_list[item[i]][0], pos_list[item[i]][1],
                            pos_list[item[i+1]][0], pos_list[item[i+1]][1])
    length += point_len(pos_list[item[-1]][0], pos_list[item[-1]][1],
                        pos_list[item[0]][0], pos_list[item[0]][1])

    return length

# -------------------------------------------------------------------
def create_hamiltonian(point_num, dlength, d_min, param_c):
    # Hamiltonian of the distance between two points
    # Spin definition
    spin_x = Array.create('x', shape=(point_num, point_num), vartype='BINARY')  # (Turn, City)

    # Set the combination of points (Excluding the same place)
    pairs = []
    for val1, val2 in product(range(point_num), range(point_num)):
        if val1 != val2:
            pairs.append((val1, val2))

    ha_d = 0
    for (j, k) in pairs:
        ha_d += sum((dlength[j][k] - d_min[j]) * spin_x[i, j] * spin_x[(i+1)%point_num, k] for i in range(point_num))

    # Hamiltonian of constraint
    ha_a = 0
    ha_b = 0
    for j in range(point_num):
        ha_a += param_c * (sum(spin_x[i, j] for i in range(point_num)) - 1)**2
    for i in range(point_num):
        ha_b += param_c * (sum(spin_x[i, j] for j in range(point_num)) - 1)**2

    # Model generation and conversion to QUBO format
    return ha_d + ha_a + ha_b

def create_onehot_constraint(point_num):
    # create one hot constraint rule.
    onehot = [0] * (2 * point_num)
    for i in range(point_num):
        onehot1 = [0] * point_num
        onehot2 = [0] * point_num
        for j in range(point_num):
            onehot1[j] = 'x[%d][%d]' % (i, j)
            onehot2[j] = 'x[%d][%d]' % (j, i)
        onehot[i] = onehot1
        onehot[point_num + i] = onehot2

    return onehot

def create_fixed_spin_constraint(point_num):
    # create fixed spin constraint rule.
    fixed = {}
    for i in range(point_num):
        fixed['x[0][%d]' % i] = 0
        fixed['x[%d][0]' % i] = 0
    fixed['x[0][0]'] = 1

    return fixed

def create_model(pos_list):
    point_num = len(pos_list)

    dlength, d_min, max_len = calc_distance(pos_list, point_num)

    hamiltonian = create_hamiltonian(point_num, dlength, d_min, Placeholder('num_a'))
    model = hamiltonian.compile()

    param_c = max_len * 0.15
    qubo, offset = model.to_qubo(feed_dict={'num_a': param_c})

    onehot_constraint = create_onehot_constraint(point_num)
    fixed_constraint = create_fixed_spin_constraint(point_num)

    solve_param = NECParameterModel(
        offset=offset,
        num_reads=1,
        num_sweeps=1000,
        onehot=onehot_constraint,
        fixed=fixed_constraint,
        timeout=200
    )

    return qubo, solve_param

def tsp_lib(arguments):
    if len(arguments) == 1:
        print("No TSP file.")
        sys.exit(1)

    point_num, pos_list = read_tsp_file(arguments[1])
    print("Read ", arguments[1])

    # -------------------------------------------------------------------
    # Annealing
    qubo, solve_param = create_model(pos_list)

    model = QuboDict(qubo)

    optimizer = StrangeworksOptimizer(
        model=model,
        solver="nec.vector_annealer",
        options=solve_param
    )

    job = optimizer.run()
    results = optimizer.results(job.slug)

    # Check of result
    print('Energy,  Length')

    best_item = None
    best_length = None
    for result in results.solution_options["raw_results"]:
        try:
            item = get_item(result['spin'], point_num)
            length = calc_length(item, pos_list)
            if best_length is None or length < best_length:
                best_length = length
                best_item = item
        except ValueError:
            length = None

        print('%s, %s' %
              ('%7.3f' % result['energy'] if result['constraint'] else 'INVALID',
               '%7.3f' % length if length else '  ERROR'))

    print('----------------------------------------')
    print('[Result]')
    print('  Turn[] = ' + str(best_item))
    print('  length = ' + str(best_length))

# Run the function

In [ ]:
tsp_lib(['eil51', 'eil51.tsp'])

Read  eil51.tsp
Energy,  Length
100.000, 448.000
----------------------------------------
[Result]
  Turn[] = [0, 31, 26, 50, 45, 10, 37, 4, 48, 9, 29, 38, 32, 44, 14, 43, 36, 16, 11, 46, 17, 3, 41, 39, 18, 40, 12, 24, 13, 5, 47, 6, 42, 23, 22, 25, 7, 30, 27, 2, 35, 34, 19, 28, 20, 33, 8, 49, 15, 1, 21]
  length = 448
